In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from tensorflow.python.keras import models
from tensorflow.python.keras import layers

2023-06-07 17:16:48.365060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [87]:
allDeaths = pd.read_csv("datasets/all_deaths.csv", encoding='latin-1')
allJails = pd.read_csv("datasets/all_jails.csv", encoding='latin-1')


# https://bjs.ojp.gov/library/publications/prisoners-2021-statistical-tables#:~:text=The%20U.S.%20prison%20population%20was,decrease%20from%202011%20(1%2C599%2C000)
prisonerDemographics = pd.read_csv("datasets/p21stt01.csv", encoding='latin-1')
prisonersPerYear = pd.read_csv("datasets/p21stat02.csv", encoding='latin-1')

# https://www.kaggle.com/datasets/christophercorrea/prisoners-and-crime-in-united-states?select=crime_and_incarceration_by_state.csv
crimesAndIncarcerationsByState = pd.read_csv("datasets/crime_and_incarceration_by_state.csv", encoding='latin-1')
prisonCustodyByState = pd.read_csv("datasets/prison_custody_by_state.csv", encoding='latin-1')
crimeTotalsByState = 


In [88]:
# clean jail data

# allDeaths data frame -> might not be too useful
allDeaths = allDeaths.drop(["jail", "id", "full_name", "last_name", "first_name", "mid_name", "race_detail", "what_info_from_media", "cause_detail"], axis=1)

# allJails data frame
allJails = allJails.drop(["id", "fips", "state_notes", "jail_notes", ])



,id,fips,statecode,state,state_notes,county,jail,jail_notes,d2008,d2009,...,med2010,med2011,med2012,med2013,med2014,med2015,med2016,med2017,med2018,med2019
0,1,1003.0,AL,Alabama,NaN,Baldwin,Baldwin County Corr. Center,NaN,0.0,0.0,...,public,public,public,public,public,public,public/Quality Correctional Health Care,Quality Correctional Health Care,Quality Correctional Health Care,Quality Correctional Health Care
1,2,1015.0,AL,Alabama,NaN,Calhoun,Calhoun County Jail,NaN,0.0,1.0,...,public,public,public,public,public,public/Quality Correctional Health Care,Quality Correctional Health Care,Quality Correctional Health Care/Southern Heal...,Southern Health Partners,Southern Health Partners
2,3,1055.0,AL,Alabama,NaN,Etowah,Etowah County Detention Center,NaN,0.0,0.0,...,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,Doctors' Care Family Medical Services,NaN
3,4,1069.0,AL,Alabama,NaN,Houston,Houston County Jail,NaN,1.0,0.0,...,public,public,public,public,public,public,public,public,public,public
4,5,1073.0,AL,Alabama,NaN,Jefferson,Jefferson County Birmingham Jail,ADP for 2008 was based on data provided by the...,NaN,NaN,...,Health Assurance,Health Assurance/Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare,Advanced Correctional Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,498,56021.0,WY,Wyoming,NaN,Laramie,Laramie County Detention Facility,NaN,0.0,0.0,...,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath
519,499,56025.0,WY,Wyoming,NaN,Natrona,Natrona County Detention Center,NaN,0.0,1.0,...,public,public,public,public,public,public,public,public,public,NaN
520,500,56031.0,WY,Wyoming,NaN,Platte,Platte County Detention Center,NaN,0.0,0.0,...,public,public,public,public,public,public,public,public,public,public
521,501,56033.0,WY,Wyoming,NaN,Sheridan,Sheridan County Detention Center,NaN,0.0,0.0,...,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath,Wellpath


In [72]:
# clean prisoner data

# prisonerDemographics data frame
prisonerDemographics = prisonerDemographics.drop(range(0, 10)).reset_index(drop=True)
prisonerDemographics = prisonerDemographics.drop(range(11, 17)).reset_index(drop=True)
prisonerDemographics = prisonerDemographics.drop(["Unnamed: 1", "Unnamed: 3", "Unnamed: 5", "Unnamed: 7", "Unnamed: 9", "Unnamed: 11"], axis=1)
prisonerDemographics.columns = ["Year", "Total", "Federal", "State", "Male", "Female"]


# prisonersPerYear data frame
prisonersPerYear = prisonersPerYear.drop(range(0, 10)).reset_index(drop=True)
prisonersPerYear = prisonersPerYear.drop(range(26, 28)).reset_index(drop=True)
prisonersPerYear.columns = ["Year", "Sentenced Prisoners"]


prisonerDemographics

,Year,Total,Federal,State,Male,Female
0,2011,"1,598,968","216,362","1,382,606","1,487,561","111,407"
1,2012,"1,570,397","217,815","1,352,582","1,461,625","108,772"
2,2013,"1,576,950","215,866","1,361,084","1,465,592","111,358"
3,2014,"1,562,319","210,567","1,351,752","1,449,291","113,028"
4,2015,"1,526,603","196,455","1,330,148","1,415,112","111,491"
5,2016,"1,508,129","189,192","1,318,937","1,396,296","111,833"
6,2017,"1,489,189","183,058","1,306,131","1,377,815","111,374"
7,2018,"1,464,385","179,898","1,284,487","1,353,595","110,790"
8,2019,"1,430,165","175,116","1,255,049","1,322,256","107,909"
9,2020,"1,221,164","152,156","1,069,008","1,137,410","83,754"
